In [15]:
import gc
import json
import tarfile
import codecs
import sys 

from collections import defaultdict

In [16]:
# bad query id
bad_query_id = "1030303"

def load_prerank(file_name, collection, top_k=1000):
    prerank = defaultdict(list)
    min_rank = 999
    with open(file_name) as f:
        for line in f:
            elements = line.split(" ")
            if elements[2] in collection and len(prerank[elements[0]])<top_k:
                article = collection[elements[2]]
                prerank[elements[0]].append({"id":elements[2], 
                                              "score":elements[4],
                                              "text":article["text"],
                                              "title":article["title"]})
            
    print(min_rank)    
    
    # create test collection base on the docs
    #docs_per_topic = [len(docs_topic) for docs_topic in prerank.values()]
    #print("average docs per topic", sum(docs_per_topic)/len(docs_per_topic), "min:",min(docs_per_topic),"max:",max(docs_per_topic))
    
    return prerank

def collection_iterator(file_name, f_map=None):
    return collection_iterator_fn(file_name=file_name, f_map=f_map)()

def collection_iterator_fn(file_name, f_map=None):
    
    reader = codecs.getreader("ascii")
    tar = tarfile.open(file_name)

    print("[CORPORA] Openning tar file", file_name)

    members = tar.getmembers()
    
    def generator():
        for m in members:
            print("[CORPORA] Openning tar file {}".format(m.name))
            f = tar.extractfile(m)
            articles = json.load(reader(f))
            if f_map is not None:
                articles = list(map(f_map, articles))
            yield articles
            f.close()
            del f
            gc.collect()
    return generator

In [17]:
# load collection
collection = sum([ articles for articles in collection_iterator("/backup/MS-MARCO/ms-marco-docs.tar.gz")],[])
collection = {x["id"]:x for x in collection}

prerank = load_prerank("/backup/MS-MARCO/msmarco-doctest2020-top100", collection)

[CORPORA] Openning tar file /backup/MS-MARCO/ms-marco-docs.tar.gz
[CORPORA] Openning tar file tmp/tmpvmkbfob_/ms-marco-docs_0000000_to_0500000
[CORPORA] Openning tar file tmp/tmpvmkbfob_/ms-marco-docs_0500000_to_1000000
[CORPORA] Openning tar file tmp/tmpvmkbfob_/ms-marco-docs_1000000_to_1500000
[CORPORA] Openning tar file tmp/tmpvmkbfob_/ms-marco-docs_1500000_to_2000000
[CORPORA] Openning tar file tmp/tmpvmkbfob_/ms-marco-docs_2000000_to_2500000
[CORPORA] Openning tar file tmp/tmpvmkbfob_/ms-marco-docs_2500000_to_3000000
[CORPORA] Openning tar file tmp/tmpvmkbfob_/ms-marco-docs_3000000_to_3213834
999


In [18]:
len(prerank[bad_query_id])

100

In [20]:
prerank[bad_query_id]